# Differential Rotation

The new version of `starry` implements an experimental version of differential rotation. This is still an experimental feature, mostly because it is extremely slow to compute. Because differential rotation leads to winding of features on the surface, the angular scale of these surface features becomes smaller and smaller with time, so the actual degree of the spherical harmonic expansion we'd need to preserve these features would increase without bound. `starry` implements a truncated expansion of the effect of differential rotation, but the operations still scale very steeply with the degree of the original map. Additionally, the approximation becomes worse and worse over time. This is something fundamental about differential rotation: it's simply not possible to model it accurately with spherical harmonics.

However, for weak differential rotation ($\alpha \ll 1$) and over short timescales, the implementation in `starry` may still be useful! Let's take a look.

In [ ]:
%matplotlib inline

In [ ]:
%run notebook_setup.py

Let's import `starry` as usual, and disable lazy evaluation.

In [ ]:
import starry
import numpy as np
import matplotlib.pyplot as plt

starry.config.lazy = False
starry.config.quiet = True

To create a map with differential rotation enabled, pass the `drorder` keyword to `starry.Map()`. This is the order of the approximation to differential rotation and should usually be 1 (or 2 if absolutely necessary). In general, the spherical harmonic expansion scales *terribly* with the order.

In [ ]:
map = starry.Map(ydeg=5, drorder=1)

Let's incline the map a bit and give it a dipole-like feature:

In [ ]:
map.inc = 60
map[2, 0] = 1

Here's what that looks like in lat-lon coordinates:

In [ ]:
map.show(projection="rect")

And here's an animation over two rotation cycles (no differential rotation yet):

In [ ]:
map.show(theta=np.linspace(0, 720, 200), dpi=150)

and the corresponding light curve:

In [ ]:
theta = np.linspace(0, 720, 1000)
flux0 = map.flux(theta=theta)
plt.plot(theta, flux0);

We haven't yet provided a value for the differential rotation parameter, so there's nothing exciting going on. To do that, we specify the $\alpha$ parameter, which is the linear shear due to differential rotation:

In [ ]:
map.alpha = 0.1

Given this linear shear, the angular velocity $\omega$ at a latitude $\phi$ on the surface is given by

$\omega(\phi) = \omega_{eq}(1 - \alpha \sin^2\phi)$,

where $\omega_{eq}$ is the angular velocity at the equator.

Here's what the rotating map now looks like over two cycles. Note the shearing away from the equator:

In [ ]:
map.show(projection="rect", theta=np.linspace(0, 720, 100))

Here's the same map in projection:

In [ ]:
map.show(theta=np.linspace(0, 720, 200), dpi=150)

Finally, let's compare the two light curves:

In [ ]:
theta = np.linspace(0, 720, 1000)
flux = map.flux(theta=theta)
plt.plot(theta, flux0, label=r"$\alpha = 0$")
plt.plot(theta, flux, label=r"$\alpha = 0.1$")
plt.legend(loc="lower left");

Finally, another word of caution. Since differential rotation leads to increasingly smaller features on the surface, the `starry` approximation breaks down after a certain amount of time. The plots below show images of the surface of this star for different values of the quantity $\omega t \alpha$, in which the expansion is done in `starry`. 

Note that for small values of $\omega t \alpha$, things look great, but above about $90^\circ$, the polar regions develop unphysical "blobs". That's the limited resolution of the map kicking in: there's no easy way to accurately describe a strongly sheared profile with $l = 5$ spherical harmonics!

The blue and orange lines show the position of the peak intensity of the map, computed analytically (blue) and from the `starry` approximation (orange). The `starry` solution tracks the analytic one pretty well also up to about $90^\circ$, beyond which the limiations of the approximation becomes evident.

So, **bottom line:** when `drorder=1`, the `starry` approximation is only good up to about $\omega t \alpha = 90^\circ$.

In [ ]:
# Render the map at several phases
theta = np.linspace(0, 360 * 4, 17)
res = 300
images = map.render(projection="rect", theta=theta, res=res)
lat = np.linspace(-90, 90, res, endpoint=False)
lon = np.linspace(-180, 180, res, endpoint=False)

# Compute the longitude of the maximum on the side
# facing the observer; it's easiest if we just mask the far side
images_masked = np.array(images)
images_masked[:, :, : (res // 4)] = 0
images_masked[:, :, -(res // 4) :] = 0
lon_starry = [lon[np.argmax(img, axis=1)] for img in images_masked]

# Compute the expected longitude of the maximum given
# the linear differential rotation law
lon_exact = [-theta_i * map.alpha * np.sin(lat * np.pi / 180) ** 2 for theta_i in theta]

# Plot it!
fig, ax = plt.subplots(5, 3, figsize=(12, 12))
ax = ax.flatten()
for k in range(len(ax)):
    ax[k].imshow(
        images[k],
        extent=(-180, 180, -90, 90),
        cmap="plasma",
        vmin=images[0].min(),
        vmax=images[0].max(),
    )
    ax[k].plot(lon_exact[k], lat)
    ax[k].plot(lon_starry[k], lat)
    ax[k].set_title(r"$\omega t \alpha = {}^\circ$".format(theta[k] * map.alpha))
    ax[k].set_ylim(-75, 75)
    for tick in ax[k].xaxis.get_major_ticks() + ax[k].yaxis.get_major_ticks():
        tick.label.set_fontsize(8)